In [50]:
import dgl
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np

# Define the GCN model
class GCN(nn.Module):
    def __init__(self, in_feats, out_feats):
        super(GCN, self).__init__()
        self.conv1 = dgl.nn.GraphConv(in_feats, 16)
        self.conv2 = dgl.nn.GraphConv(16, out_feats)

    def forward(self, g, features):
        x = F.relu(self.conv1(g, features))
        x = self.conv2(g, x)
        return x

# Create a graph
def build_graph():
    g = dgl.DGLGraph()
    g.add_nodes(10)  # Initial 10 nodes
    # Add edges among the initial nodes
    src = np.random.randint(0, 10, size=30)
    dst = np.random.randint(0, 10, size=30)
    g.add_edges(src, dst)
    g = dgl.add_self_loop(g)
    return g


# Main function to run the process
    # Set device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Build the graph
g = build_graph().to(device)

# Add initial node features
node_features = torch.randn(10, 3)  # 10 nodes with 3-dimensional random features
g.ndata['feat'] = node_features.to(device)

# Initialize the model
net = GCN(3, 5).to(device)  # 3-dimensional to 5-dimensional features
optimizer = torch.optim.Adam(net.parameters(), lr=0.01)

# Train the model
for epoch in range(50):
    net.train()
    logits = net(g, g.ndata['feat'])
    logp = F.log_softmax(logits, 1)
    target = torch.LongTensor([i % 5 for i in range(10)]).to(device)  # Random targets for demo
    loss = F.nll_loss(logp, target)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    print(f'Epoch {epoch} | Loss: {loss.item()}')

Epoch 0 | Loss: 1.6383445262908936
Epoch 1 | Loss: 1.6191332340240479
Epoch 2 | Loss: 1.6009435653686523
Epoch 3 | Loss: 1.5838308334350586
Epoch 4 | Loss: 1.5674822330474854
Epoch 5 | Loss: 1.5525562763214111
Epoch 6 | Loss: 1.538368821144104
Epoch 7 | Loss: 1.5249302387237549
Epoch 8 | Loss: 1.5120902061462402
Epoch 9 | Loss: 1.4997704029083252
Epoch 10 | Loss: 1.4882854223251343
Epoch 11 | Loss: 1.4773728847503662
Epoch 12 | Loss: 1.4668142795562744
Epoch 13 | Loss: 1.456069827079773
Epoch 14 | Loss: 1.445007085800171
Epoch 15 | Loss: 1.4335291385650635
Epoch 16 | Loss: 1.421600580215454
Epoch 17 | Loss: 1.4097126722335815
Epoch 18 | Loss: 1.397286057472229
Epoch 19 | Loss: 1.3842493295669556
Epoch 20 | Loss: 1.3706670999526978
Epoch 21 | Loss: 1.3566410541534424
Epoch 22 | Loss: 1.3424216508865356
Epoch 23 | Loss: 1.3279674053192139
Epoch 24 | Loss: 1.3130663633346558
Epoch 25 | Loss: 1.297823190689087
Epoch 26 | Loss: 1.282370924949646
Epoch 27 | Loss: 1.2665859460830688
Epoch 28 

In [51]:
new_node_count = 3
node_features = g.ndata['feat']
old_node_count = g.number_of_nodes()
old_node_count

10

In [59]:
g.nodes(), g.edges()

(tensor([0, 1, 2, 3, 4, 5, 6, 7, 8, 9], device='cuda:0'),
 (tensor([9, 0, 0, 4, 3, 2, 0, 1, 7, 1, 4, 4, 8, 0, 1, 8, 7, 2, 5, 4, 6, 9, 9, 6,
          4, 7, 4, 7, 8, 7, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9], device='cuda:0'),
  tensor([3, 3, 1, 4, 2, 0, 9, 2, 2, 4, 4, 6, 4, 9, 5, 8, 3, 2, 0, 6, 0, 0, 6, 1,
          9, 1, 1, 2, 2, 1, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9], device='cuda:0')))

In [48]:
new_nodes = old_node_count + new_node_count
g.add_nodes(new_node_count)
new_src = np.random.randint(0, new_nodes, size=10)
new_dst = np.random.randint(0, new_nodes, size=10)
g.add_edges(new_src, new_dst)
g = dgl.add_self_loop(g)
# Adding new random features for new nodes
new_features = torch.randn(new_node_count, node_features.shape[1]).to(device)
all_features = torch.cat([node_features, new_features], 0)

# Get embeddings for the updated graph
new_embeddings = net(g, all_features)

# Simulate adding new nodes and updating their embeddings
# new_embeddings = add_and_embed_nodes(net, g, 3, g.ndata['feat'])
print('New node embeddings:', new_embeddings[-3:])